The training dataset is opened as a Pandas Dataframe. 

In [1]:
import pandas as pd
import numpy as np
train_data = pd.read_csv('Train_Set.csv')

ModuleNotFoundError: No module named 'pandas'

The dependent variables are split into 2 categories, healthy and any form of cancer. Healthy outputs are renamed as 0 and the rest as 1. However, the proportion of the training data which represents the healthy population is severely outmatched by the proportion that represents the sample population with cancer. This may affect the model such that the accuracy and precision of the model is overinflated.

In [ ]:
train_data['class_label'] = train_data['class_label'].replace(['healthy', 'early stage cancer', 'screening stage cancer', 'mid stage cancer', 'late stage cancer'], [0, 1, 1, 1, 1])
# converting all cancer patients to 1 and healthy patients to 0
print(train_data['class_label'])
print(train_data.iloc[:, -1])
print(train_data['class_label'].value_counts())
# class_label
# 1    2133
# 0      60

0       0
1       0
2       0
3       0
4       0
       ..
2188    1
2189    1
2190    1
2191    1
2192    1
Name: class_label, Length: 2193, dtype: int64
0       0
1       0
2       0
3       0
4       0
       ..
2188    1
2189    1
2190    1
2191    1
2192    1
Name: class_label, Length: 2193, dtype: int64
class_label
1    2133
0      60
Name: count, dtype: int64


/tmp/ipykernel_5943/3360202232.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_data['class_label'] = train_data['class_label'].replace(['healthy', 'early stage cancer', 'screening stage cancer', 'mid stage cancer', 'late stage cancer'], [0, 1, 1, 1, 1])


The RandomUnderSampler from the 'imblearn' package is used to randomly undersample the majority class, which is the proportion of the population with the cancer, such that the ratio of healthy to cancer is 1:1.

In [ ]:
x, y = train_data.drop(['class_label'], axis=1), train_data['class_label']
from imblearn.under_sampling import RandomUnderSampler
undersample = RandomUnderSampler(sampling_strategy = 1)
X_over, y_over = undersample.fit_resample(x, y)

Gaussian Naive Bayes classifier is used, since the input variables are continuous. One assumption we make is that the values follow a normal distrubution. 

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(X_over, y_over)
test_data = pd.read_csv('Test_Set.csv')
test_data['class_label'] = test_data['class_label'].replace(['healthy', 'early stage cancer', 'screening stage cancer', 'mid stage cancer', 'late stage cancer'], [0, 1, 1, 1, 1])
testX, testY = test_data.drop(['class_label'], axis=1), test_data['class_label']
testX_over, testY_over = undersample.fit_resample(testX, testY)
nb.score(testX_over, testY_over)

/tmp/ipykernel_5943/794660813.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_data['class_label'] = test_data['class_label'].replace(['healthy', 'early stage cancer', 'screening stage cancer', 'mid stage cancer', 'late stage cancer'], [0, 1, 1, 1, 1])


0.6829268292682927

The summary of the Gaussian Naive Bayes classifier is as shown below.

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
y_pred = nb.predict(testX_over)
print(classification_report(testY_over, y_pred))
confusion_matrix(testY_over, nb.predict(testX_over))

              precision    recall  f1-score   support

           0       0.63      0.88      0.73        41
           1       0.80      0.49      0.61        41

    accuracy                           0.68        82
   macro avg       0.72      0.68      0.67        82
weighted avg       0.72      0.68      0.67        82



array([[36,  5],
       [21, 20]])